In [116]:
import pandas as pd
from sc_target_evidence_utils import cxg_utils, cellontology_utils, plotting_utils

In [92]:
import numpy as np
from scipy import sparse
import cellxgene_census

census = cellxgene_census.open_soma(census_version="2023-05-15")

In [84]:
disease_ontology_id = 'MONDO:0005115'

In [93]:
cxg_cell_metadata = pd.read_csv('/nfs/team205/ed6/data/cellxgene_hsapiens_cell_metadata.csv')

/tmp/ipykernel_4867/1181248227.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  cxg_cell_metadata = pd.read_csv('/nfs/team205/ed6/data/cellxgene_hsapiens_cell_metadata.csv')


In [97]:
## Read cxg cleaned metadata
output_dir = '/nfs/team205/ed6/bin/sc_target_evidence/data/'
cxg_metadata = pd.read_csv(output_dir + 'cellxgene_hsapiens_donor_metadata.disease_relevant_annotation.csv', index_col=0)

In [130]:
cxg_metadata.groupby(['tissue_general']).nunique('dataset_id').sort_values('dataset_id')

,assay,tissue,suspension_type,disease,dataset_id,donor_id,development_stage_ontology_term_id,sample_id,disease_name_original,disease_relevant_tissue,tissue_general_original,disease_ontology_id_original,disease_ontology_id
tissue_general,,,,,,,,,,,,,
abdomen,1,1,1,1,1,5,4,5,1,1,1,1,1
urinary bladder,1,1,1,1,1,1,1,1,1,1,1,1,1
paracolic gutter,1,1,1,1,1,1,1,1,1,1,1,1,1
parietal peritoneum,1,1,1,1,1,1,1,1,1,1,1,1,1
peritoneum,1,1,1,1,1,14,11,14,1,1,1,1,1
placenta,2,1,1,1,1,11,1,11,1,1,1,1,1
pleura,1,1,1,1,1,1,1,1,1,1,1,1,1
lamina propria,2,1,1,1,1,5,3,5,1,1,1,1,1
reproductive system,1,1,1,1,1,3,1,3,1,1,1,1,1


In [112]:
## Get datasets of interest
sc_metadata = cxg_utils._get_cxg_datasets(cxg_metadata, disease_ontology_id=disease_ontology_id, keep_disease_relevant_tissue=True, keep_normal=True)
dataset_ids = sc_metadata['dataset_id'].unique()
tissue_ids = sc_metadata['tissue_general_original'].unique()
disease_ids = sc_metadata['disease'].unique()


# Get annotations 
cxg_cell_metadata = pd.read_csv('/nfs/team205/ed6/data/cellxgene_hsapiens_cell_metadata.csv')
cxg_cell_metadata = cxg_cell_metadata[(cxg_cell_metadata.tissue_general.isin(tissue_ids)) & (cxg_cell_metadata.disease.isin(disease_ids)) & (cxg_cell_metadata.dataset_id.isin(dataset_ids))].copy()

## Clean cell ontologies
print("Cleaning cell ontologies...")
graph = cellontology_utils.get_cellontology_graph(output_dir)

# Exclude cell types with less than 5 cells
ct_counts = cell_metadata["cell_type_ontology_term_id"].value_counts()
ontology_terms = ct_counts[ct_counts > 5].index.tolist()
ct_rename_dict = cellontology_utils.rename_cts_to_high_level(ontology_terms, graph)


In [119]:
cell_metadata["high_level_cell_type_ontology_term_id"] = [
    ct_rename_dict[x] if x in ct_rename_dict.keys() else 'low_quality_annotation' for x in cell_metadata["cell_type_ontology_term_id"] 
    ]


In [120]:
plotting_utils.plot_celltype_rename(cell_metadata, graph)

KeyError: 'count'